In [134]:
import pandas as pd
import joblib

In [135]:
logistic = pd.read_csv('../predictions/LogisticRegression.csv')
dt = pd.read_csv('../predictions/DecisionTreeClassifier.csv')
rf = pd.read_csv('../predictions/RandomForestClassifier.csv')
xgb = pd.read_csv('../predictions/XGBClassifier.csv')
grid = pd.read_csv('../predictions/GridSearchCV.csv')

In [136]:
logistic.head()

,Unnamed: 0,Loan_ID,Loan_Status,Gender,Race
0,0,LP001015,1,Male,Black
1,1,LP001022,1,Male,Black
2,2,LP001031,1,Male,Black
3,3,LP001051,1,Male,Black
4,4,LP001054,1,Male,Black


In [137]:
races = logistic.Race.unique()
genders = logistic.Gender.unique()
races, genders

(array(['Black', 'White'], dtype=object),
 array(['Male', 'Female'], dtype=object))

In [138]:
logistic['Loan_Status'].value_counts()

Loan_Status
1    348
0     19
Name: count, dtype: int64

### Demographic parity

In [139]:
def demo_parity(df: pd.DataFrame, col: str, val: str):
    total_num = len(df[df[col] == val])
    predict_true = len(df[(df['Loan_Status'] == 1) & (df[col] == val)])
    return predict_true/total_num

In [140]:
class Model:
    def __init__(self, name: str):
        self.name = name
        self.gender_ratio = 0
        self.race_ratio = 0
        self.fair_score = 0

    def __str__(self):
        return f"Model('{self.name}', fair_score={self.fair_score})"


In [141]:
model_list = [Model('Logistic'), Model('DecisionTree'), Model('RandomForest'), 
              Model('XGB'), Model('GridSearch')]

In [142]:
def show_bias(df: pd.DataFrame, model: Model):
    demo_dict = {}
    print('Gender: ')
    for g in genders:
        demo_dict[g] = demo_parity(df, 'Gender', g)
        print(g, demo_dict[g])

    difference = abs(demo_dict["Male"] - demo_dict["Female"])
    ratio = demo_dict["Male"]/demo_dict["Female"] if demo_dict["Female"] != 0 else 0
    ratio = ratio if ratio < 1 else 1/ratio
    model.gender_ratio = ratio
    print('Difference: ', difference)
    print('Ratio: ', ratio)

    print()
    demo_dict = {}
    print('Race: ')
    for r in races:
        demo_dict[r] = demo_parity(df, 'Race', r)
        print(r, demo_dict[r])

    difference = abs(demo_dict["White"] - demo_dict["Black"])
    ratio = demo_dict["White"]/demo_dict["Black"] if demo_dict["Black"] != 0 else 0
    ratio = ratio if ratio < 1 else 1/ratio
    model.race_ratio = ratio
    print('Difference: ', difference)
    print('Ratio: ', ratio)

### Base Model: Logistic Regression

In [143]:
show_bias(logistic, model_list[0])

Gender: 
Male 0.9595959595959596
Female 0.9
Difference:  0.059595959595959536
Ratio:  0.9378947368421052

Race: 
Black 0.9760479041916168
White 0.925
Difference:  0.051047904191616755
Ratio:  0.9476993865030675


In [144]:
show_bias(dt, model_list[1])

Gender: 
Male 0.632996632996633
Female 0.5571428571428572
Difference:  0.07585377585377584
Ratio:  0.8801671732522798

Race: 
Black 0.7005988023952096
White 0.55
Difference:  0.1505988023952095
Ratio:  0.7850427350427351


In [145]:
show_bias(rf, model_list[2])

Gender: 
Male 0.7676767676767676
Female 0.6428571428571429
Difference:  0.12481962481962472
Ratio:  0.837406015037594

Race: 
Black 0.8143712574850299
White 0.685
Difference:  0.12937125748502987
Ratio:  0.841139705882353


In [146]:
show_bias(xgb, model_list[3])

Gender: 
Male 0.6868686868686869
Female 0.6571428571428571
Difference:  0.02972582972582971
Ratio:  0.9567226890756302

Race: 
Black 0.7544910179640718
White 0.62
Difference:  0.1344910179640718
Ratio:  0.8217460317460318


In [147]:
show_bias(grid, model_list[4])

Gender: 
Male 0.7676767676767676
Female 0.7142857142857143
Difference:  0.05339105339105332
Ratio:  0.9304511278195491

Race: 
Black 0.8323353293413174
White 0.695
Difference:  0.13733532934131742
Ratio:  0.835


#### Gender

In [148]:
model_list = sorted(model_list, key=lambda x: x.gender_ratio)
for model in model_list:
    print(f'{model.name}: gender score = {model.gender_ratio}')

RandomForest: gender score = 0.837406015037594
DecisionTree: gender score = 0.8801671732522798
GridSearch: gender score = 0.9304511278195491
Logistic: gender score = 0.9378947368421052
XGB: gender score = 0.9567226890756302


#### Race

In [149]:
model_list = sorted(model_list, key=lambda x: x.race_ratio)
for model in model_list:
    print(f'{model.name}: race score = {model.race_ratio}')

DecisionTree: race score = 0.7850427350427351
XGB: race score = 0.8217460317460318
GridSearch: race score = 0.835
RandomForest: race score = 0.841139705882353
Logistic: race score = 0.9476993865030675


Here we use the average of `Race` and `Gender` demographic parity ratio as the bias score. The higher the score, the less biased the model is. 

In [150]:
for model in model_list:
    model.fair_score = (model.gender_ratio + model.race_ratio) / 2

In [151]:
model_list = sorted(model_list, key=lambda x: x.fair_score)
for model in model_list:
    print(model)

print('avg fair score: ', sum([model.fair_score for model in model_list])/len(model_list))

Model('DecisionTree', fair_score=0.8326049541475075)
Model('RandomForest', fair_score=0.8392728604599735)
Model('GridSearch', fair_score=0.8827255639097745)
Model('XGB', fair_score=0.889234360410831)
Model('Logistic', fair_score=0.9427970616725864)
avg fair score:  0.8773269601201346


In [152]:
model_list = sorted(model_list, key=lambda x: x.name)
print('Race: ')
for model in model_list:
    print(f'{model.name}: {model.race_ratio:.4f}') 

avg_race_score  = sum([model.race_ratio for model in model_list])/len(model_list)
print('avg race score: ', avg_race_score)

print('\nGender: ')
for model in model_list:
    print(f'{model.name}: {model.gender_ratio:.4f}')

avg_gender_score  = sum([model.gender_ratio for model in model_list])/len(model_list)
print('avg gender score: ', avg_gender_score)

Race: 
DecisionTree: 0.7850
GridSearch: 0.8350
Logistic: 0.9477
RandomForest: 0.8411
XGB: 0.8217
avg race score:  0.8461255718348376

Gender: 
DecisionTree: 0.8802
GridSearch: 0.9305
Logistic: 0.9379
RandomForest: 0.8374
XGB: 0.9567
avg gender score:  0.9085283484054317


Logistic regression performs best in the demographic parity test.